In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from nltk.tokenize import word_tokenize, sent_tokenize
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

from gensim.models import Word2Vec

from sklearn.feature_extraction.text import CountVectorizer

# Any results you write to the current directory are saved as output.

In [ ]:
loc = '/kaggle/input/flickr-image-dataset/flickr30k_images/flickr30k_images'

csv_loc = '/kaggle/input/flickr-image-dataset/flickr30k_images/flickr30k_images/results.csv'
images_loc = '../input/flickr-image-dataset/flickr30k_images/flickr30k_images/flickr30k_images/'

In [ ]:
imgs_df = pd.read_csv(csv_loc, sep="|")
col = imgs_df[" comment"]
col.dropna(inplace=True)

col = col.apply(lambda line : "startseq " + line.lower() + " endseq")
tok_col = col.apply(word_tokenize)

In [ ]:
tok_col[:5]

In [ ]:
wv_mod = Word2Vec(tok_col)

In [ ]:
wv_mod.wv.most_similar(["guy"], ["girl"])

In [ ]:
wv_mod.wv[tok_col.iloc[0]].shape

In [ ]:
tok_col.iloc[0]

In [ ]:
vectorizer = CountVectorizer(max_features=5000)

In [ ]:
vectorizer.fit(col)

In [ ]:
# vectorizer.vocabulary_

In [ ]:
list_lines = []
for line in tok_col:
    tok_line = [vectorizer.vocabulary_[word] for word in line if word in vectorizer.vocabulary_]
    list_lines.append(tok_line)

In [ ]:
len(vectorizer.vocabulary_)

In [ ]:
from keras.applications import ResNet50

In [ ]:
res_mod = ResNet50()

In [ ]:
# res_mod.summary()

from keras.models import Model
from keras.preprocessing.image import load_img
from keras.layers import Dense, Input, Embedding, LSTM
from keras.layers.merge import add

In [ ]:
feature_model = Model(res_mod.input, res_mod.layers[-2].output)

In [ ]:
imgs_df.columns

In [ ]:
pil_img = load_img(images_loc+imgs_df.iloc[0, 0], target_size=(224, 224))

In [ ]:
np_img = np.array(pil_img)

In [ ]:
feature_model.predict(np.array([np_img]))

In [ ]:
name_set = set(imgs_df['image_name'])

In [ ]:
img_featues = {}

for img_name in name_set:
    if img_name not in img_featues:
        pil_img = load_img(images_loc+img_name, target_size=(224, 224))
        np_img = np.array(pil_img)
        features = feature_model.predict(np.array([np_img]))
        img_featues[img_name] = features

In [ ]:
# existing caption to target word
in_layer = Input(shape=(50,))
emb_layer = Embedding(5000, 100)(in_layer)
lstm_layer = LSTM(256)(emb_layer)

In [ ]:
in_img_features = Input(shape=(2048,))
d1 = Dense(256, activation="relu")(in_img_features)
d2 = add([lstm_layer, d1])
out = Dense(5000, activation="softmax")(d2)

In [ ]:
model = Model([in_layer, in_img_features], [out])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
batch_size = 50

items = 0

while True:
    for index in range(len(imgs_df)):
        
    img_name = imgs_df["image_name"].iloc[index]
    line = list_lines[index]
    
    x1 = img_featues[img_name]
    
    for i in range(1, len(line)):
        x2 = line[:i]
        y = line[i]
        
    